### Capstone 1 - Washington state linkage of infant birth and death records

##### Maya Bhat-Gregerson

January 07, 2020

### A. PREPARATION OF BIRTH DATA, 2016-2018

### I. Data acquisition

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc

pd.set_option('display.max_columns', None)

I use SQL queries to get the birth variables I am likely to need for linking the records for 2016 through 2018. In the SQL query I limited the records to individuals born in Washington State and those who were Washington State residents.

NOTE: I renamed all variables from birth records so that they begin with 'b' to distinguish the fields from those in death records with the same names.

In [2]:
## CONNECT TO WHALES & USE SQL QUERY FOR DEATH DATA SET

driver = '{SQL Server Native Client 11.0}'

conn = pyodbc.connect(
        Trusted_Connection='Yes',
        Driver='{ODBC Driver 13 for SQL Server}',
        Server='###',
        Database='###'
        )

querystring = ("SELECT SFN_NUM as 'bsfn'," + 
        "SUBSTRING(SFN_NUM, 11, 1) as 'bcerttype'," +
        "ISNULL(CHILD_GNAME, 'NaN') as 'bfname', " +
        "ISNULL(CHILD_MNAME, 'NaN') as 'bmname', " +
        "ISNULL(CHILD_LNAME, 'NaN') as 'blname', " +
        "ISNULL(MOTHER_GNAME_PRIOR, 'NaN') as 'bmom_fname', " +
        "ISNULL(MOTHER_MNAME_PRIOR, 'NaN') as 'bmom_mname', " +
        "ISNULL(MOTHER_LNAME, ' ') as 'bmom_lname'," +       
        "ISNULL(MOTHER_LNAME, 'NaN') as 'bmom_maiden', " +
        "ISNULL(FATHER_GNAME, 'NaN') as 'bdad_fname', " +
        "ISNULL(FATHER_MNAME, 'NaN') as 'bdad_mname', " + 
        "ISNULL(FATHER_LNAME, 'NaN') as 'bdad_lname', " +       
        "ISNULL(INFANT_SEX, 'NaN') as 'bsex', " + 
		"IDOB as 'bdob', " + 
		"ISNULL(SUBSTRING(IDOB, 1,2), '99') as 'bdobm', " + 
		"ISNULL(SUBSTRING(IDOB, 4,2), '99') as 'bdobd', " +
		"ISNULL(SUBSTRING(IDOB, 7,4), '9999') as 'bdoby', " + 
		"ISNULL(BIRTH_FAC_STATE_FIPS_CD, '  ') as 'bbirplstatefips', " + 
		"ISNULL(RES_CITY, '  ') as 'b_momrescity', " + 
		"RIGHT('00000' + ISNULL(RES_CITY_FIPS_CD, '99999'), 5) as 'b_momrescityfips', " + 
		"ISNULL(RES_COUNTY, '  ') as 'b_momrescountyl', " + 
		"RIGHT('000' + ISNULL(RES_COUNTY_FIPS_CD, '999'), 3) as 'b_momrescntyfips', " + 
		"ISNULL(RES_STATE_FIPS_CD, '  ') as 'b_momresstatefips', " + 
		"ISNULL(SUBSTRING(RES_ZIP, 1,5), '99999') as 'b_momreszip' " + 
"FROM [wa_vrvweb_events].[VRV_BIRTH_TBL] " +
"WHERE ((DATE_BIRTH_YEAR = 2016) OR (DATE_BIRTH_YEAR = 2017) OR (DATE_BIRTH_YEAR = 2018))" +
	"AND FL_CURRENT = '1'" +
	"AND FL_VOIDED = '0'" +
    "AND FL_FILED <> 'N'" + 
    "AND (BIRTH_FAC_STATE_FIPS_CD = 'WA' OR RES_STATE_FIPS_CD = 'WA')")
bir1618 = pd.read_sql_query(querystring, conn)


## SAVE DATA AS CSV FILE

bir1618.to_csv(r'###\Data\raw\bir1618_raw.csv', index=None, header=True)

In [24]:
bir1618 = pd.read_csv(r'###\Data\raw\bir1618_raw.csv', low_memory = False)
bir1618.shape

(267747, 20)

In [25]:
bir1618.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267747 entries, 0 to 267746
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   bsfn               267747 non-null  object
 1   bcerttype          267747 non-null  object
 2   bfname             267732 non-null  object
 3   bmname             241846 non-null  object
 4   blname             267731 non-null  object
 5   bmom_fname         267579 non-null  object
 6   bmom_mname         224870 non-null  object
 7   bmom_lname         266052 non-null  object
 8   bsex               267747 non-null  object
 9   bdob               267747 non-null  object
 10  bdobm              267747 non-null  int64 
 11  bdobd              267747 non-null  int64 
 12  bdoby              267747 non-null  int64 
 13  bbirplstatefips    267747 non-null  object
 14  b_momrescity       267747 non-null  object
 15  b_momrescityfips   267747 non-null  int64 
 16  b_momrescountyl    2

In [56]:
#bir1618.head()

### II. Data cleaning and standardization

 - Standardize the merging ID number variable ('bsfn') so that it is an integer.

bir1618.head() shows that 'bsfn' is a string consisting of 10 numbers followed by R, O, D, or B.  I will remove the last character and then convert the remaining string (all numbers) into an integer.

In [27]:
bir1618.bsfn= bir1618.bsfn.str.rstrip('R')
bir1618.bsfn= bir1618.bsfn.str.rstrip('O')
bir1618.bsfn= bir1618.bsfn.str.rstrip('D')
bir1618.bsfn= bir1618.bsfn.str.rstrip('B')

In [28]:
bir1618['bsfn'] = bir1618['bsfn'].astype(int)

In [29]:
bir1618.bsfn.dtypes

dtype('int32')

In [30]:
bir1618.bsfn.head()

0    2016050347
1    2016046482
2    2016082422
3    2016042756
4    2016087575
Name: bsfn, dtype: int32

 - Check date of birth year ('bdoby') variable to make sure we have only the years of interest

In [31]:
bir1618['bdoby'].value_counts(dropna=False)

2016    91760
2017    88707
2018    87280
Name: bdoby, dtype: int64

 - Check to see if mother's residence state (which is considered to be the infant's resident state) is WA only.

In [32]:
bir1618['b_momresstatefips'].value_counts(dropna=False)

WA    264109
OR      1643
ID      1356
AK       188
CA        66
MT        54
ZZ        53
BC        27
XX        26
TX        20
VA        15
AZ        15
NC        12
UT        12
NV        11
HI        11
FL        10
NY         9
IL         9
TN         8
OH         7
CO         7
MI         6
MN         6
GA         5
WI         5
AL         4
MD         4
AR         4
KS         3
LA         3
MO         3
NE         3
ND         3
PA         3
KY         3
SC         3
IN         2
NJ         2
WY         2
NM         2
MA         2
AS         2
ON         1
IA         1
RI         1
VT         1
MS         1
SD         1
NH         1
CT         1
NS         1
Name: b_momresstatefips, dtype: int64

   **LIMIT DATA SET TO WASHINGTON STATE**

The following steps assure that the records include only births occurring in WA where the mother's residence state is also WA.

In [33]:
bir1618['bbirplstatefips'].value_counts(dropna=False)

WA    263547
OR      2846
ID       882
CA       143
TX        36
AZ        21
CO        18
FL        16
UT        14
MT        14
OH        12
NY        11
NV        10
AK         9
PA         9
VA         9
LA         8
MA         8
HI         8
KS         8
IL         7
NC         7
WY         7
XX         6
IN         6
NM         6
MN         6
MI         6
AR         6
CT         6
DC         5
WI         5
MO         5
GA         4
AL         4
SC         4
ND         3
TN         3
NJ         3
NE         3
MS         2
MD         2
WV         2
RI         2
OK         2
SD         2
NH         1
IA         1
DE         1
KY         1
Name: bbirplstatefips, dtype: int64

In [34]:
b1618 = bir1618[(bir1618['b_momresstatefips']=="WA")]
b1618 = b1618[(b1618['bbirplstatefips']=="WA")]

b1618['b_momresstatefips'].value_counts(dropna=False)

WA    259909
Name: b_momresstatefips, dtype: int64

In [35]:
b1618['bbirplstatefips'].value_counts(dropna=False)

WA    259909
Name: bbirplstatefips, dtype: int64

In [36]:
b1618.shape

(259909, 20)

#### CHECK FOR NULL VALUES

In [37]:
# checking for all missing variables
b1618.isna().sum()

bsfn                     0
bcerttype                0
bfname                   8
bmname               25504
blname                  13
bmom_fname              99
bmom_mname           42105
bmom_lname            1472
bsex                     0
bdob                     0
bdobm                    0
bdobd                    0
bdoby                    0
bbirplstatefips          0
b_momrescity             0
b_momrescityfips         0
b_momrescountyl          0
b_momrescntyfips         0
b_momresstatefips        0
b_momreszip              0
dtype: int64

- Mothers' and infants' middle names may not useful for linking as there are too many missing values.

 **CHECK FOR OUT OF RANGE VALUES**

In [38]:
# create dictionary of valid values so that each variable can be checked to make sure there is no
# out of range value.

valids = {'sex': ['M', 'F', 'U'],
          'dobm': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 99],
          'dobd': np.r_[1:32 ,99],
          'doby': [2016,2017,2018],
         'rcntyfips': np.r_[range(1, 78, 2), 99],
         'certtype': ['R'],
         'birthstatefips': ['WA'], 
         'rstatefips': ['WA']}

In [39]:
# check for out of range values for 'bsex'

chkbsex = b1618['bsex'].isin(valids['sex'])
len(b1618[~chkbsex])

0

In [40]:
# check for out of range values for 'bdobm'

chkbdobm = b1618['bdobm'].isin(valids['dobm'])
len(b1618[~chkbdobm])

0

In [41]:
# check for out of range values for 'bdoby'

chkbdoby = b1618['bdoby'].isin(valids['doby'])
len(b1618[~chkbdoby])

0

In [42]:
# check for out of range values for 'bdobd'

chkbdobd = b1618['bdobd'].isin(valids['dobd'])
len(b1618[~chkbdobd])

0

In [43]:
# check for out of range values for 'b_momrescntyfips'

chkbrcounty = b1618['b_momrescntyfips'].isin(valids['rcntyfips'])
len(b1618[~chkbrcounty])

133

In [44]:
# create dataframe 'brcntyerrors' which shows only rows where mom's residence county does not have a valid FIPS code

brcntyerrors = b1618[~chkbrcounty][['b_momrescntyfips', 'b_momrescountyl','b_momresstatefips']]

brcntyerrors

,b_momrescntyfips,b_momrescountyl,b_momresstatefips
104,999,,WA
2974,999,UNKNOWN,WA
3117,999,,WA
6708,999,,WA
7813,999,,WA
...,...,...,...
261290,999,UNKNOWN,WA
261693,999,UNKNOWN,WA
262797,999,UNKNOWN,WA
266689,999,BENTON,WA


In [45]:
#Create dictionary of Washington State county names and county FIPS codes 

counties = {'ADAMS':1,
'ASOTIN':3,
'BENTON':5,
'CHELAN':7,
'CLALLAM':9,
'CLARK':11,
'COLUMBIA':13,
'COWLITZ':15,
'DOUGLAS':17,
'FERRY':19,
'FRANKLIN':21,
'GARFIELD':23,
'GRANT':25,
'GRAYS HARBOR':27,
'ISLAND':29,
'JEFFERSON':31,
'KING':33,
'KITSAP':35,
'KITTITAS':37,
'KLICKITAT':39,
'LEWIS':41,
'LINCOLN':43,
'MASON':45,
'OKANOGAN':47,
'PACIFIC':49,
'PEND OREILLE':51,
'PIERCE':53,
'SAN JUAN':55,
'SKAGIT':57,
'SKAMANIA':59,
'SNOHOMISH':61,
'SPOKANE':63,
'STEVENS':65,
'THURSTON':67,
'WAHKIAKUM':69,
'WALLA WALLA':71,
'WHATCOM':73,
'WHITMAN':75,
'YAKIMA':77
}


In [46]:
# replace FIPS codes for mother's residence county code ('b_momrcntyfips') variable by mapping values from dictionary 'counties' based on 
# the literal name of the county ('b_momrcountyl').  ONLY DO FOR out of range or missing values for WA counties.

b1618.loc[b1618['b_momresstatefips']=='WA','b_momrescntyfips']=b1618['b_momrescountyl'].map(counties)

In [47]:
# recheck to see if there are fewer with out of range values
chkbrcounty = b1618['b_momrescntyfips'].isin(valids['rcntyfips'])
len(b1618[~chkbrcounty][b1618['b_momresstatefips']=='WA'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


125

In [48]:
# recheck residence county literal to see where there are problems
b1618['b_momrescountyl'].value_counts(dropna=False)

KING            75449
PIERCE          34421
SNOHOMISH       29629
SPOKANE         17601
CLARK           14972
YAKIMA          11398
THURSTON         9350
KITSAP           9076
BENTON           7845
WHATCOM          6597
FRANKLIN         4716
GRANT            4401
SKAGIT           4320
COWLITZ          3675
ISLAND           2742
LEWIS            2715
CHELAN           2622
GRAYS HARBOR     2216
MASON            1943
CLALLAM          1907
WALLA WALLA      1869
DOUGLAS          1558
OKANOGAN         1482
STEVENS          1293
KITTITAS         1230
WHITMAN          1150
ADAMS            1148
JEFFERSON         542
PEND OREILLE      339
PACIFIC           317
LINCOLN           310
SAN JUAN          244
FERRY             215
SKAMANIA          138
COLUMBIA          102
KLICKITAT          94
ASOTIN             75
UNKNOWN            63
                   62
WAHKIAKUM          60
GARFIELD           23
Name: b_momrescountyl, dtype: int64

In [49]:
# recheck number of missing in 'b_momrescntyfips'
b1618['b_momrescntyfips'].isna().sum()

125

 - The 125 records with missing residence county FIPS codes had either blanks or "unknown" in the residence county literal field.  The residence county literal field is used to look up the FIPS codes and populate 'b_momrescntyfips'.  Without any additional information it is not possible to find and include the correct values. There is no **easy** way to find mom's county of residence (literal or code).  Will leave these in data for now.

#### STANDARDIZE STRING VARIABLES

First, middle, and last names of infants and mothers as well as city names will be standardized by converting these columns to upper case text, removing white spaces, removing hyphens and other punctuation marks.

In [57]:
#b1618.tail()

In [51]:
#convert all string variables to upper case
b1618 = b1618.apply(lambda x: x.str.upper() if type(x) == str else x)

In [52]:
# remove white spaces on either side of name or within name; remove punctuation
b1618 = b1618.apply(lambda x: x.str.strip() if type(x) == str else x)
b1618 = b1618.applymap(lambda x: x.replace(" ", "") if type(x) == str else x)
b1618 = b1618.applymap(lambda x: x.replace("-", "") if type(x) == str else x)
b1618 = b1618.applymap(lambda x: x.replace(".", "") if type(x) == str else x)
b1618 = b1618.applymap(lambda x: x.replace("'", "") if type(x) == str else x)

In [58]:
#Check with .tail() to see if string transformations were successful
#b1618.tail(30)

In [54]:
b1618.bdob = pd.to_datetime(b1618.bdob)

A check using .tail() before and after the string transformations showed that change to uppercase, removal of punctuation marks and white spaces were all completed successfully.

In [55]:
b1618.to_csv(r'Y:\DQSS\Death\MBG\Py\Data\clean\b1618_clean.csv', index=None, header=True)

Next step - do the same for death data for 2016-18.